## Short Term Wind Climate 

In this file the short term wind climate is processed with the data organized in the 01 and 02 files. 
For this reason the following steps are done in this file: 

16. Inport Libaries, load datapaths , Load datapats int pd. DataFrame 
17. Calculate the monthly and annual wind statistics for the windspeed and the winddirection for both met masts 
18. Plot the monthly mean in windspeed and winddrirection for both buoys 
19. Plot windroses for both buays 
20. group the data into hours of the day and calcukate the mean windspeed and winddirection of every day 
21. plot the diurnal windspeed and winddirection for both buoys 
22. perform some checks about the grouping 
23. Plot a Weilbul distribuntion of the windspeed 
24. Calculate the annual Power Production of one Turbine, one field and the entrie farm 
25. Plot a power curve 



#### 16. Import libraries, load data paths, load data paths into a pandas DataFrame

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
import pandas as pd
import netCDF4 as nc
from netCDF4 import Dataset
import numpy as np
from scipy.stats import linregress
from matplotlib.projections.polar import PolarAxes
from windrose import WindroseAxes
from scipy.stats import weibull_min
from scipy import interpolate
import scipy.integrate as integrate


In [ ]:
# Data Paths
interpolated_csv_path = 'interpolated_ws_and_wd_for_150_m.csv'
turbine_power_curve_path = 'data/turbine-info/power_curves/IEA-15MW-D240-H150.csv'

In [ ]:
#Load the data into pandas dataframes 
df_interpol_height = pd.read_csv(interpolated_csv_path)
df_interpol_height.set_index('time', inplace=True)

df_interpol_height

#### 17. Calculate the monthly and annual wind statistics for the wind speed and the wind direction for both buoys

In [ ]:
def divide_into_bins_and_calculate_statistics(data_ws, data_wd, num_bins):
    # Calculate the size of each bin
    bin_size = len(data_ws) // num_bins
    
    # Initialize lists to store the means and standard deviations of each bin
    bin_means_ws = []
    bin_stdv_ws = []
    bin_means_wd = []
    bin_stdv_wd = []

    # Iterate through each bin
    for i in range(num_bins):
        # Determine the start and end indices of the bin
        start_idx = i * bin_size
        end_idx = start_idx + bin_size
        
        # Extract the bin data for wind speed and wind direction
        bin_data_ws = data_ws[start_idx:end_idx]
        bin_data_wd = data_wd[start_idx:end_idx]
        
        # Calculate the mean and standard deviation of the bin for wind speed
        bin_mean_ws = np.mean(bin_data_ws)
        bin_std_ws = np.std(bin_data_ws)
        
        # Calculate the mean and standard deviation of the bin for wind direction
        bin_mean_wd = np.mean(bin_data_wd)
        bin_std_wd = np.std(bin_data_wd)
        
        # Append the mean and standard deviation to their respective lists
        bin_means_ws.append(bin_mean_ws)
        bin_stdv_ws.append(bin_std_ws)
        bin_means_wd.append(bin_mean_wd)
        bin_stdv_wd.append(bin_std_wd)

    # Create a DataFrame from the lists
    df = pd.DataFrame({
        'Monthly Means WS': bin_means_ws,
        'Monthly Stdv WS': bin_stdv_ws,
        'Monthly Means WD': bin_means_wd,
        'Monthly Stdv WD': bin_stdv_wd
    })

    return df

# Calculate monthly statistics
monthly_stats_ws6 = divide_into_bins_and_calculate_statistics(df_interpol_height['ws6_150m'], df_interpol_height['wd6_150m'], 12)
monthly_stats_ws2 = divide_into_bins_and_calculate_statistics(df_interpol_height['ws2_150m'], df_interpol_height['wd2_150m'], 12)

# Print the resulting DataFrames
print("Monthly Statistics for Buoy 6:")
print(monthly_stats_ws6)

print("Monthly Statistics for Buoy 2:")
print(monthly_stats_ws2)


In [ ]:
# Calculate yearly statistics for wind speed
yearly_mean_ws6 = np.mean(df_interpol_height['ws6_150m'])
yearly_std_ws6 = np.std(df_interpol_height['ws6_150m'])
yearly_mean_ws2 = np.mean(df_interpol_height['ws2_150m'])
yearly_std_ws2 = np.std(df_interpol_height['ws2_150m'])

print(f"Yearly Mean of Wind Speed Buoy 6: {yearly_mean_ws6:.2f}, Standard Deviation Buoy 6: {yearly_std_ws6:.2f}")
print(f"Yearly Mean of Wind Speed Buoy 2: {yearly_mean_ws2:.2f}, Standard Deviation Buoy 2: {yearly_std_ws2:.2f}")

# Calculate yearly statistics for wind direction
yearly_mean_wd6 = np.mean(df_interpol_height['wd6_150m'])
yearly_std_wd6 = np.std(df_interpol_height['wd6_150m'])
yearly_mean_wd2 = np.mean(df_interpol_height['wd6_150m'])
yearly_std_wd2 = np.std(df_interpol_height['wd6_150m'])

print(f"Yearly Mean of Wind Direction Buoy 6: {yearly_mean_wd6:.2f}, Standard Deviation Buoy 6: {yearly_std_wd6:.2f}")
print(f"Yearly Mean of Wind Direction Buoy 2: {yearly_mean_wd2:.2f}, Standard Deviation Buoy 2: {yearly_std_wd2:.2f}")


#### 18. Plot the monthly mean wind speed and wind direction for both buoys

In [ ]:
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
# Extract the monthly means from the dataframe
monthly_means_ws6 = monthly_stats_ws6['Monthly Means WS']
monthly_means_wd6 = monthly_stats_ws6['Monthly Means WD']

# Plot the monthly mean wind speeds and wind directions
fig, ax1 = plt.subplots(figsize=(10, 6))

bar_width = 0.4  # Set bar width to half of the current width

# Bar plot for wind speeds
ax1.bar(months, monthly_means_ws6, color='blue', edgecolor='black', label='Mean Wind Speed', width=bar_width)
ax1.set_xlabel('Month')
ax1.set_ylabel('Mean Wind Speed (m/s)', color='blue')
ax1.set_ylim(0, max(monthly_means_ws6) + 2)  # Adjust y-axis limit for better visualization
ax1.tick_params(axis='y', labelcolor='blue')
ax1.grid(True, axis='y', linestyle='--', alpha=0.7)

# Create a secondary y-axis
ax2 = ax1.twinx()
ax2.plot(months, monthly_means_wd6, color='red', marker='o', linestyle='-', label='Mean Wind Direction')
ax2.set_ylabel('Mean Wind Direction (°)', color='red')
ax2.tick_params(axis='y', labelcolor='red')
ax2.set_ylim(0, 360)  # Assuming wind direction is in degrees

fig.tight_layout()
plt.title('Monthly Mean Wind Speeds and Directions for Buoy 6')
plt.show()


In [ ]:
# Extract the monthly means from the dataframe
monthly_means_ws2 = monthly_stats_ws2['Monthly Means WS']
monthly_means_wd2 = monthly_stats_ws2['Monthly Means WD']

# Plot the monthly mean wind speeds and wind directions
fig, ax1 = plt.subplots(figsize=(10, 6))

bar_width = 0.4  # Set bar width to half of the current width

# Bar plot for wind speeds
ax1.bar(months, monthly_means_ws2, color='blue', edgecolor='black', label='Mean Wind Speed', width=bar_width)
ax1.set_xlabel('Month')
ax1.set_ylabel('Mean Wind Speed (m/s)', color='blue')
ax1.set_ylim(0, max(monthly_means_ws2) + 2)  # Adjust y-axis limit for better visualization
ax1.tick_params(axis='y', labelcolor='blue')
ax1.grid(True, axis='y', linestyle='--', alpha=0.7)

# Create a secondary y-axis
ax2 = ax1.twinx()
ax2.plot(months, monthly_means_wd2, color='red', marker='o', linestyle='-', label='Mean Wind Direction')
ax2.set_ylabel('Mean Wind Direction (°)', color='red')
ax2.tick_params(axis='y', labelcolor='red')
ax2.set_ylim(0, 360)  # Assuming wind direction is in degrees

fig.tight_layout()
plt.title('Monthly Mean Wind Speeds and Directions for Buoy 2')
plt.show()


#### 19. Plot wind roses for both buoys

In [ ]:

def plot_wind_rose(wd, ws, title):
    ax = WindroseAxes.from_ax()
    ax.bar(wd, ws, normed=True, opening=0.8, edgecolor='white')
    ax.set_legend()
    plt.title(title)
    plt.show()

plot_wind_rose(df_interpol_height['wd6_150m'], df_interpol_height['ws6_150m'], 'Windrose Plot of Buoy 6 from 03-03-2022 to 03-03-2023' )
plot_wind_rose(df_interpol_height['wd2_150m'], df_interpol_height['ws2_150m'], 'Windrose Plot of Buoy 2 from 03-03-2022 to 03-03-2023' )

#### 20. Group the data into hours of the day and calculate the mean wind speed and wind direction for each hour of the day
#### 21. Plot the diurnal wind speed and wind direction for both buoys

In [ ]:
# Group by hour and calculate the mean wind speed and direction for each hour
df_interpol_height.index = pd.to_datetime(df_interpol_height.index)
df_interpol_height['hour'] = df_interpol_height.index.hour
diurnal_profile = df_interpol_height.groupby('hour').mean()

# Plot for Buoy 6
fig, ax1 = plt.subplots(figsize=(12, 6))

# Plot wind speed histogram on the left y-axis
ax1.bar(diurnal_profile.index, diurnal_profile['ws6_150m'], width=0.4, label='Wind Speed 150m (Buoy 6)', color='b', align='center')
ax1.set_xlabel('Hour of the Day')
ax1.set_ylabel('Wind Speed (m/s)', color='b')
ax1.tick_params(axis='y', labelcolor='b')

# Create a second y-axis to plot wind direction
ax2 = ax1.twinx()
ax2.plot(diurnal_profile.index, diurnal_profile['wd6_150m'], 'r--o', label='Wind Direction 150m (Buoy 6)', markersize=5)
ax2.set_ylabel('Wind Direction (degrees)', color='r')
ax2.tick_params(axis='y', labelcolor='r')

# Add legends
fig.legend(loc="upper right", bbox_to_anchor=(1,1), bbox_transform=ax1.transAxes)

# Title and grid
plt.title('Diurnal Profile of Wind Speed and Wind Direction for Buoy 6')
plt.grid(True)

plt.show()

# Plot for Buoy 2
fig, ax1 = plt.subplots(figsize=(12, 6))

# Plot wind speed histogram on the left y-axis
ax1.bar(diurnal_profile.index, diurnal_profile['ws2_150m'], width=0.4, label='Wind Speed 150m (Buoy 2)', color='b', align='center')
ax1.set_xlabel('Hour of the Day')
ax1.set_ylabel('Wind Speed (m/s)', color='b')
ax1.tick_params(axis='y', labelcolor='b')

# Create a second y-axis to plot wind direction
ax2 = ax1.twinx()
ax2.plot(diurnal_profile.index, diurnal_profile['wd2_150m'], 'r--o', label='Wind Direction 150m (Buoy 2)', markersize=5)
ax2.set_ylabel('Wind Direction (degrees)', color='r')
ax2.tick_params(axis='y', labelcolor='r')

# Add legends
fig.legend(loc="upper right", bbox_to_anchor=(1,1), bbox_transform=ax1.transAxes)

# Title and grid
plt.title('Diurnal Profile of Wind Speed and Wind Direction for Buoy 2')
plt.grid(True)

plt.show()


#### 22. Perform some checks on the grouping

In [ ]:
#check if groupby does what it should manually 
# df_hourly = df_interpol_height.resample('h').mean()
# df_filtered1hour = df_hourly[df_hourly.index.strftime('%H:%M:%S') == '00:00:00'].mean()
# print(df_filtered1hour)
# print(diurnal_profile)

#print the 'hour' group to see that it marks one hour and then the next and so on 
# pd.set_option('display.max_rows', None)
# print(df_interpol_height['hour'])


#### 23. Plot a Weibull distribution of the wind speed

- weibull_min.fit(ws_data, floc=0): This part of the code fits a Weibull distribution to the wind speed data (ws_data). The fit method of the weibull_min distribution estimates the shape, location, and scale parameters of the Weibull distribution that best fit the provided data.
- shape, _, scale: The result of the fit method is a tuple containing the estimated parameters. In this case, shape represents the shape parameter of the Weibull distribution, and scale represents the scale parameter. The underscore _ is used to discard the estimated location parameter (floc), as it is fixed at 0 in this case.
- The weibull_pdf function defines the probability density function (PDF) for a Weibull distribution. function that describes the likelihood of a continuous random variable falling within a particular range of values.

In [ ]:
# Function to fit Weibull distribution and plot
def plot_weibull_fit(ws, title):
    # Fit Weibull distribution
    params = weibull_min.fit(ws, floc=0)# floc=0 => location parameter defaults to 0
    shape, loc, scale = params
    x = np.linspace(0, ws.max(), 100)
    weibull_pdf = weibull_min.pdf(x, shape, loc, scale)
    
    # Plot histogram and Weibull fit
    plt.figure()
    plt.hist(ws, bins=30, density=True, alpha=0.6, color='g', edgecolor='black')
    plt.plot(x, weibull_pdf, 'r-', label=f'Weibull fit: shape={shape:.2f}, scale={scale:.2f}')
    plt.title(title)
    plt.xlabel('Wind Speed (m/s)')
    plt.ylabel('Density')
    plt.legend()
    plt.show()


#### 24. Calculate the annual power production of one turbine, one field, and the entire farm

Remark on wind turbine data: The "ct" in the turbine data refers to the thrust coefficient. It is a dimensionless number that describes the thrust force exerted by the wind on the turbine blades relative to the dynamic pressure of the wind.

In [ ]:
# Constants
T = 8760  # total hours/year [h]
rho = 1.225  # air density [kg/m^3]
D = 240  # rotor diameter [m]
A = np.pi * (D / 2)**2  # swept area [m^2]

# Load data
# Replace 'df_interpol_height' with the actual variable name for your measured wind speed data
# Assuming 'ws6_150m' is a column in 'df_interpol_height' DataFrame
windspeed_data = df_interpol_height['ws6_150m']
power_curve_data = pd.read_csv(turbine_power_curve_path)

def cut_in_windspeed(power_curve_data):
    return power_curve_data.loc[power_curve_data['P'] > 0, 'ws'].min()

def cut_out_windspeed(power_curve_data):
    return power_curve_data.loc[power_curve_data['P'] > 0, 'ws'].max()

def power_curve_interpolated(power_curve_data):
    return interpolate.interp1d(power_curve_data['ws'], power_curve_data['P'], fill_value="extrapolate")

def calculate_weibull_fit(windspeed_data):
    shape, _, scale = weibull_min.fit(windspeed_data, floc=0)  # floc=0 => location parameter defaults to 0
    return shape, scale

def weibull_pdf(ws, shape, scale):
    return weibull_min.pdf(ws, shape, loc=0, scale=scale)

def integrand(U, shape, scale, power_curve_func):
    P = power_curve_func(U)  # Use interpolated power curve values
    return P * weibull_pdf(U, shape, scale)

def calculate_APP(shape, scale, power_curve_func, cut_in_ws, cut_out_ws):
    APP, error = integrate.quad(integrand, cut_in_ws, cut_out_ws, args=(shape, scale, power_curve_func), limit=100, epsabs=1e-05, epsrel=1e-05)
    return APP * T, error * T  # Multiply by total hours per year to get AEP

# Process data
cut_in_ws = cut_in_windspeed(power_curve_data)
cut_out_ws = cut_out_windspeed(power_curve_data)
power_curve_func = power_curve_interpolated(power_curve_data)
shape, scale = calculate_weibull_fit(windspeed_data)

# Calculate APP (Annual Power Production)
APP, error = calculate_APP(shape, scale, power_curve_func, cut_in_ws, cut_out_ws)
print(f"APP of one Turbine: {APP / 1e6:.4f} GWh")  # Convert to MWh for readability
print(f"Estimated error: {error / 1e6:.4f} GWh")

turbines_area_of_interest_path = 'data/turbine-info/coordinates/area_of_interest/'
base_path = turbines_area_of_interest_path
file_N9_1 = f'{base_path}\layout-N-9.1.geom.csv'
file_N9_2 = f'{base_path}\layout-N-9.2.geom.csv'
file_N9_3 = f'{base_path}\layout-N-9.3.geom.csv'

# Load the data
data_N9_1 = pd.read_csv(file_N9_1)
data_N9_2 = pd.read_csv(file_N9_2)
data_N9_3 = pd.read_csv(file_N9_3)

print(f"Annual Energy Production of N-9.1 (133 Turbines): {((APP/1e9) * len(data_N9_1)):.4f} TWh")
print(f"Annual Energy Production of N-9.2 (133 Turbines): {((APP/1e9) * len(data_N9_2)):.4f} TWh")
print(f"Annual Energy Production of N-9.3 (100 Turbines): {((APP/1e9) * len(data_N9_3)):.4f} TWh")
print(f"Total Energy Production of all three fields (366 Turbines): {((APP/1e9) *366):.4f} TWh ")
print(f"This is {((((APP/1e9) *366)/507)*100):.2f} % of the electricity consumed in one yr in Germany.")

#### 25. Plot a power curve

In [ ]:
# Plotting the power curve
plt.figure(figsize=(10, 6))
plt.plot(power_curve_data['ws'], power_curve_data['P'], marker='o', linestyle='-', color='b')
plt.title('Power Curve')
plt.xlabel('Wind Speed (m/s)')
plt.ylabel('Power (kW)')
plt.grid(True)
plt.show()